In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import * #for window

spark = SparkSession\
.builder\
.master("yarn")\
.appName("spark bucketing")\
.enableHiveSupport()\
.config("spark.shuffle.useOldFetchProtocol",'true')\
.config("spark.sql.warehouse.dir","/user/itv009490/warehouse")\
.getOrCreate()

spark

In [2]:
order_schema = 'order_id  long, order_date string, customer_id long, order_status string '
orders_df = spark.read.format("csv").\
schema(order_schema).\
load("/public/trendytech/orders/orders_1gb.csv")

In [3]:
customers_schema = """customerid long, customer_fname string, customer_lname string, username string, password string,
address string, city string, state string, pincode long"""
customers_df = spark.read.format("csv").schema(customers_schema).load("/public/trendytech/retail_db/customers")

In [4]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold','-1')

In [6]:
orders_df.write.mode("overwrite").bucketBy(8,"customer_id").sortBy("customer_id").\
option("path","/user/itv009490/orders101").saveAsTable("orderstab101")

In [9]:
spark.sql("desc formatted orderstab101").show(truncate=False)

+----------------------------+-----------------------------------------------------------+-------+
|col_name                    |data_type                                                  |comment|
+----------------------------+-----------------------------------------------------------+-------+
|order_id                    |bigint                                                     |null   |
|order_date                  |string                                                     |null   |
|customer_id                 |bigint                                                     |null   |
|order_status                |string                                                     |null   |
|                            |                                                           |       |
|# Detailed Table Information|                                                           |       |
|Database                    |default                                                    |       |
|Table    

In [10]:
customers_df.write.mode("overwrite").bucketBy(8,"customerid").sortBy("customerid").\
option("path","/user/itv009490/customers101").saveAsTable("customers101")

In [11]:
orders_df.rdd.getNumPartitions()

9

In [12]:
customers_df.rdd.getNumPartitions()

1

In [13]:
spark.sql("""select * from orderstab101 join customers101 on orderstab101.customer_id == customers101.customerid
""").write.format("noop").mode("overwrite").save()